# Google Colab으로 오픈소스 LLM 구동하기

## 1단계 - LLM 양자화에 필요한 패키지 설치
- bitsandbytes: Bitsandbytes는 CUDA 사용자 정의 함수, 특히 8비트 최적화 프로그램, 행렬 곱셈(LLM.int8()) 및 양자화 함수에 대한 경량 래퍼
- PEFT(Parameter-Efficient Fine-Tuning): 모델의 모든 매개변수를 미세 조정하지 않고도 사전 훈련된 PLM(언어 모델)을 다양한 다운스트림 애플리케이션에 효율적으로 적용 가능
- accelerate: PyTorch 모델을 더 쉽게 여러 컴퓨터나 GPU에서 사용할 수 있게 해주는 도구


In [1]:
#양자화에 필요한 패키지 설치
# %%capture
# !pip install -U bitsandbytes
# !pip install -U git+https://github.com/huggingface/transformers.git
# !pip install -U git+https://github.com/huggingface/peft.git
# !pip install -U git+https://github.com/huggingface/accelerate.git

## 2단계 - 트랜스포머에서 BitsandBytesConfig를 통해 양자화 매개변수 정의하기


* load_in_4bit=True: 모델을 4비트 정밀도로 변환하고 로드하도록 지정
* bnb_4bit_use_double_quant=True: 메모리 효율을 높이기 위해 중첩 양자화를 사용하여 추론 및 학습
* bnd_4bit_quant_type="nf4": 4비트 통합에는 2가지 양자화 유형인 FP4와 NF4가 제공됨. NF4 dtype은 Normal Float 4를 나타내며 QLoRA 백서에 소개되어 있습니다. 기본적으로 FP4 양자화 사용
* bnb_4bit_compute_dype=torch.bfloat16: 계산 중 사용할 dtype을 변경하는 데 사용되는 계산 dtype. 기본적으로 계산 dtype은 float32로 설정되어 있지만 계산 속도를 높이기 위해 bf16으로 설정 가능



In [20]:
import torch
import warnings
warnings.filterwarnings('ignore')
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## 3단계 - 경량화 모델 로드하기

이제 모델 ID를 지정한 다음 이전에 정의한 양자화 구성으로 로드합니다.

In [3]:
# %%capture
# ! pip install accelerate

In [21]:
model_id = "kyujinpy/Ko-PlatYi-6B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(78464, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
  

## 4단계 - 잘 실행되는지 확인

In [24]:
device = "cuda:0"
messages = [
    {"role": "user", "content": "딥러닝 대해 알려줘"}
]
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds.to(device)
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<|startoftext|> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

딥러닝 대해 알려줘 [/INST]<<SYS>><|endoftext|>


## 5단계- RAG 시스템 결합하기

In [25]:
# pip install시 utf-8, ansi 관련 오류날 경우 필요한 코드
import locale
import warnings
warnings.filterwarnings('ignore')

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [26]:
# !pip -q install langchain pypdf chromadb sentence-transformers faiss-gpu
# !pip install langchain_community

In [28]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=300,
)

prompt_template = """
### [INST]
Instruction: Answer the question based on your knowledge.
Here is context to help:
{context}
### QUESTION:
{question}
[/INST]
 """

koplatyi_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=koplatyi_llm, prompt=prompt)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

### 한 파일로 처리하는 방법

## **핵심정리**

In [13]:
'''
loader = PyPDFLoader("/teamspace/studios/this_studio/조상구장군.pdf")

pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)
'''

'\nloader = PyPDFLoader("/teamspace/studios/this_studio/조상구장군.pdf")\n\npages = loader.load_and_split()\ntext_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)\ntexts = text_splitter.split_documents(pages)\n'

- pdf로 만든다는 것 알겠는데 아래처럼 생긴 데이터를 document object를 어떻게 만드는지 알려줘? 
- 엑셀(test)의 정보제공컬럼까지 copy하여 chatGPT에게 물어보고
- Document는 page_content, metadata의 속성을 갖고 그런데 metadata에는 국가명, 수집일자, 제목, 내용, 정보제공국가로 구성된다. 내용이 바로 page_content이다.

- pdf로 만든다는 것 알겠는데 아래처럼 생긴 데이터를 document object를 어떻게 만드는지 알려줘? 
- 엑셀(test)의 정보제공컬럼까지 copy하여 chatGPT에게 물어보고
- Document는 page_content, metadata의 속성을 갖고 그런데 metadata에는 국가명, 수집일자, 제목, 내용, 정보제공국가로 구성된다. 내용이 바로 page_content이다.

### **기사를 보고 원인요소를 예측**
#### 1. Data Preparation
- documents list contains both the page_content and the 식품등 유형 in its metadata.
- label each document with the correct "식품등 유형".
##### 1.1. 훈련데이터 준비(2020년 ~ 2022년) 
- 전체 엑셀파일 읽어서 피클 gzip을 압축하여 'Combined_Food_Safety_Data_2015_2023_gzip.pkl'로 저장

In [53]:
'''
import pandas as pd
import glob
# Use glob to get all file paths matching the pattern for the years 2020 to 2022
file_paths = glob.glob('Untitled Folder/*.xlsx')
file_paths

# Initialize an empty list to store dataframes
dfs = []
ttl = 0
# Read each Excel file and append to the list
for file_path in file_paths:
    df = pd.read_excel(file_path)
    df['수집일자'] = pd.to_datetime(df['수집일자'])
    ttl += df.shape[0]
    file_name = file_path.split('/')[1].split('.')[0]
    print(f'{file_name}: 데이터크기 {df.shape}, 누적크기는 {ttl}')
    dfs.append(df)
# Concatenate all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)
combined_df

pd.to_datetime(df['수집일자'])

# # Save the combined dataframe to a new Excel file
output_file_path = 'Untitled Folder/Combined_Food_Safety_Data_2015_2023_gzip.pkl'
combined_df.to_pickle(output_file_path, compression='gzip')

print(f"Combined file saved to {output_file_path}")

'''


식품안전정보DB(2020): 데이터크기 (19607, 8), 누적크기는 19607
식품안전정보DB(2021): 데이터크기 (23135, 8), 누적크기는 42742
식품안전정보DB(2022): 데이터크기 (22233, 8), 누적크기는 64975
식품안전정보DB(2023): 데이터크기 (20775, 8), 누적크기는 85750
식품안전정보DB(2019): 데이터크기 (19457, 8), 누적크기는 105207
식품안전정보DB(2016): 데이터크기 (17339, 8), 누적크기는 122546
식품안전정보DB(2017): 데이터크기 (18711, 8), 누적크기는 141257
식품안전정보DB(2018): 데이터크기 (18950, 8), 누적크기는 160207
식품안전정보DB(2015): 데이터크기 (16273, 8), 누적크기는 176480
Combined file saved to Untitled Folder/Combined_Food_Safety_Data_2015_2023_gzip.pkl


In [59]:
import pandas as pd
# Assuming your DataFrame `df` has a column '식품등유형'
file_path = 'Untitled Folder/Combined_Food_Safety_Data_2015_2023_gzip.pkl'  # Replace with your actual file path
df_all = pd.read_pickle(file_path, compression='gzip')  # Update with the correct file path
df_all.head()

,국가명,수집일자,제목,내용,정보제공국가,정보구분,식품등유형,원인요소
0,미국,2020-12-31,"[확대회수]미국 Flowers Foods, Inc., 글루텐 함유 가능성으로 빵 2...","*기수집정보: [회수]미국 Flowers Foods, Inc., 글루텐 함유 가능성...",미국,위해식품정보,"가공식품>과자류, 빵류 또는 떡류>빵류",표시광고>원료·성분·함량>원료·성분·함량
1,중국,2020-12-31,"중국, 부적합 식품 생산한 'Wuhan Wansongtang Bio-medical ...",1)- 생산기업: Wuhan Wansongtang Bio-medical techn...,중국,위해식품정보,가공식품>음료류>다류,표시광고>원료·성분·함량>원료·성분·함량
2,홍콩,2020-12-31,"홍콩 식품안전센터, 일본산 식품 검사 일일 개황('20년 12월 30일 정오 12시...",식품환경위생서 식품안전센터는 2011년 3월 12일부터 일본의 대지진 이후 발생한 ...,일본,글로벌 동향정보,식품일반>식품일반>식품일반,신규 위해요소>방사능>방사능
3,중국,2020-12-31,"중국 광둥성 중산시, 냉동 닭날개 외포장 핵산 검사 양성","12월 29일, 광둥(广东)성 중산(中山)시 둥펑(东凤)진에서 냉동식품에 대한 자발...",중국,글로벌 동향정보,축산물>식육류>닭고기,생물학적 위해요소>기타>기타(신종코로나바이러스)
4,국제기구,2020-12-31,"세계동물보건기구, 고병원성 조류 인플루엔자 동물 감염 현황(12.30)","1. 우크라이나, H5 (즉각보고서) - 최초발생일: 2020.12.28 - 발생보...",제외국,글로벌 동향정보,축산물>축산물일반>축산물일반,생물학적 위해요소>동식물질병>동물질병


#### 훈련데이터는 2015년부터 2023년 6월31일까지

In [61]:
df = df_all[df_all['수집일자'] <= '2023-06-30']

In [62]:
# Define a custom Document class
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

    def __repr__(self):
        content_preview = str(self.page_content) if pd.notna(self.page_content) else 'No Content'
        return f"Document(page_content={content_preview}..., metadata={self.metadata})"

# Create documents from the DataFrame
documents = [
    Document(
        page_content=row['내용'],
        metadata={
            '국가명': row['국가명'],
            '수집일자': row['수집일자'],
            '제목': row['제목'],
            '원인요소': row['원인요소']  # Ensure this column exists in your data
        }
    )
    for _, row in df.iterrows()
]

# Prepare data for training
data = [(doc.page_content, doc.metadata['원인요소']) for doc in documents if '원인요소' in doc.metadata]
df_data = pd.DataFrame(data, columns=['page_content', '원인요소'])

# Convert labels to integers
label_mapping = {label: idx for idx, label in enumerate(df_data['원인요소'].unique())}
df_data['label'] = df_data['원인요소'].map(label_mapping)
df_data


,page_content,원인요소,label
0,"*기수집정보: [회수]미국 Flowers Foods, Inc., 글루텐 함유 가능성...",표시광고>원료·성분·함량>원료·성분·함량,0
1,1)- 생산기업: Wuhan Wansongtang Bio-medical techn...,표시광고>원료·성분·함량>원료·성분·함량,0
2,식품환경위생서 식품안전센터는 2011년 3월 12일부터 일본의 대지진 이후 발생한 ...,신규 위해요소>방사능>방사능,1
3,"12월 29일, 광둥(广东)성 중산(中山)시 둥펑(东凤)진에서 냉동식품에 대한 자발...",생물학적 위해요소>기타>기타(신종코로나바이러스),2
4,"1. 우크라이나, H5 (즉각보고서) - 최초발생일: 2020.12.28 - 발생보...",생물학적 위해요소>동식물질병>동물질병,3
...,...,...,...
165130,"22일, 광둥(广东)성 식품약품감독관리국은 2015년 제1차 성급 식품 샘플검사 정...",기타>기타>기타,3739
165131,청두(成都）에서 부동산 중개업을 하는 53세 여성 류(刘)씨는 작년 5월 이웃 상인...,안전위생>안전>안전,7
165132,후생노동성은 '2015년도 수입식품 감시지도계획'을 책정하여 30일 공표함. 수입식...,안전위생>안전>안전,7
165133,"51개 회원국, 1개 회원 기구(EU) 및 31개 국제기구 대표 260여 명이 참석...",해당없음>해당없음>해당없음,77


### 2. Model Training
- We'll use a simple text classification model. For this example, we'll use transformers from Hugging Face.

In [63]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch

# Define the dataset class
class FoodTypeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(label_mapping))

# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(df_data['page_content'].tolist(), df_data['label'].tolist(), test_size=0.1, random_state=42)

# Create the datasets
train_dataset = FoodTypeDataset(train_texts, train_labels, tokenizer, max_len=128)
val_dataset = FoodTypeDataset(val_texts, val_labels, tokenizer, max_len=128)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Create the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,8.592500
20,8.568500
30,8.582000
40,8.549500
50,8.527100
60,8.496600
70,8.451800
80,8.305400
90,8.266300
100,8.126400


ValueError: Input nan is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

### 3. Prediction
- To make predictions with the trained model, we need to encode the new text data and use the model to predict the label.

In [17]:
import torch

# Function to predict the "원인요소" for new articles
def predict_food_type(page_content):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    inputs = tokenizer.encode_plus(
        page_content,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()

    # Map the integer prediction back to the label
    label_map = {v: k for k, v in label_mapping.items()}
    predicted_label = label_map[prediction]

    return predicted_label

# Example usage
new_article = '''
호주 언론 'News.com.au'에 따르면, 서호주(West Australia)주 보건당국이 Qukes 브랜드의 오이(baby cucumber) 제품을 섭취한 
사람들에게서 설사 증세가 발생하자 해당 제품을 긴급 회수 조치한다고 발표했음. 크리스마스 기간 동안 12명 이상이 해당 오이를 섭취한 후 아팠음. 
12월 23일 퍼스(Perth)의 한 슈퍼마켓에서 구입한 오이에서 살모넬라 티피무리움(Salmonella Typhimurium)이 검출되었음.  
주 보건당국은 관련 조사에 착수했으며, 2022년 12월에 구입한 제품을 섭취하지 않도록 당부했음. 
언론은 다른 브랜드의 오이는 현재까지 영향을 받지 않았다고 보도했음.
'''
predicted_type = predict_food_type(new_article)
print(f"The predicted 원인요소 is: {predicted_type}")



The predicted 원인요소 is: 생물학적 위해요소>미생물>살모넬라(Salmonella spp.)
